In [1]:
import os
import pandas as pd
import requests

ModuleNotFoundError: No module named 'pandas'

In [6]:
DATA_DIR = "data"
SUBMISSIONS_DIR = "submissions"

In [30]:
assert os.path.isdir(DATA_DIR)
assert os.path.isdir(SUBMISSIONS_DIR)

# 1. Loading the input data set

In [1]:
DATA_SET_NAME = "dataset.tsv"

In [5]:
data_set = pd.read_csv(f"{DATA_DIR}/{DATA_SET_NAME}", delimiter="\t", names=["city1", "city2", "latency", "cost", "messages"])
data_set.head(-1)

NameError: name 'pd' is not defined

# 2. Scoring Function

This scoring function is ready to use and grades your submission, it returns the same score as what the Latency Challenge website would return. **Note:**, this doesn't actually submit your solution! You can use this as a utility but as long as you don't upload a solution this will not put you on the leaderboard.

In [65]:
def score_submission(submission: set[tuple[str, str]]) -> (float, str):
    """
    This is the scoring function you can use to determine how well your solution is doing
    """

    cities = set(data_set["city1"]) | set(data_set["city2"])
    edges_available = set(
        tuple(sorted(x))
        for _, x in data_set[data_set["latency"] > 0][["city1", "city2"]].iterrows()
    )
    edges_available = set((x, y) if x < y else (y, x) for (x, y) in edges_available)

    city_to_index = {city: i for i, city in enumerate(cities)}
    max_latency = 10_000
    distances = [[10 * max_latency] * len(cities) for _ in range(len(cities))]
    messages = [[0] * len(cities) for _ in range(len(cities))]

    edges_cost = 0
    for _, row in data_set.iterrows():
        i1 = city_to_index[row["city1"]]
        i2 = city_to_index[row["city2"]]

        print((row["city1"], row["city2"]), submission)
        if (row["city1"], row["city2"]) in submission or (row["city2"], row["city1"]) in submission:
            edges_cost += row["cost"]
            distances[i1][i2] = row["latency"]
            distances[i2][i1] = row["latency"]

        messages[i1][i2] = row["messages"]
        messages[i2][i1] = row["messages"]

    for j in range(len(cities)):
        for i in range(len(cities)):
            for k in range(len(cities)):
                distances[i][k] = min(distances[i][k], distances[i][j] + distances[j][k])

    profit = 0

    for i in range(len(cities)):
        for j in range(len(cities)):
            score_per_message = max(0, max_latency - distances[i][j])
            profit += score_per_message * messages[i][j]

    profit //= 2

    return profit - edges_cost

# 3. Running your solver

In [6]:
SOLVER_NAME = "all_edges"

In [11]:
solver_data_set = data_set.copy()
solver_data_set.head()

NameError: name 'data_set' is not defined

In [12]:
# this is an example solver that simply picks all edges
submission = solver_data_set[
    (solver_data_set["latency"] != 0) & (solver_data_set["cost"] != 0)
].drop(columns=["latency", "cost", "messages"])
submission.head()

def find(parent, x):
    if parent[x] != x:
        parent[x] = find(parent, parent[x])  # Path compression
    return parent[x]

def union(parent, rank, x, y):
    root_x = find(parent, x)
    root_y = find(parent, y)
    if root_x != root_y:
        if rank[root_x] > rank[root_y]:
            parent[root_y] = root_x
        elif rank[root_x] < rank[root_y]:
            parent[root_x] = root_y
        else:
            parent[root_y] = root_x
            rank[root_x] += 1

# 2. Generate MST (Kruskal's Algorithm)
def generate_mst(data):
    edges = [
        (row["cost"], row["latency"], row["city1"], row["city2"])
        for _, row in data.iterrows()
        if row["cost"] > 0 and row["latency"] > 0
    ]
    
    edges.sort()  # Sort by cost
    
    cities = set(data["city1"]).union(set(data["city2"]))
    parent = {city: city for city in cities}
    rank = {city: 0 for city in cities}

    mst = []
    for cost, latency, city1, city2 in edges:
        if find(parent, city1) != find(parent, city2):
            union(parent, rank, city1, city2)
            mst.append((city1, city2))

    return mst

# 3. Additional Edge Selection (Optional)
def add_additional_edges(data, mst):
    mst_set = set(mst)
    additional_edges = []

    for _, row in data.iterrows():
        edge = (row["city1"], row["city2"])
        if edge not in mst_set and edge[::-1] not in mst_set:
            # Add edges with high latency-to-cost ratio
            ratio = (row["messages"] * max(0, 10_000 - row["latency"])) / row["cost"] if row["cost"] > 0 else 0
            if ratio > 1:  # Threshold to ensure cost-effectiveness
                additional_edges.append(edge)

    return additional_edges

NameError: name 'solver_data_set' is not defined

# 5. Score your submission

In [82]:
submission_records = list(map(tuple, submission.to_records(index=False)))
score_submission(submission_records)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



214000085

This score should be equal to the score you get when submitting to the tech challenge website.

# 5. Submit your solution

In [83]:
submission.to_csv(f"{SUBMISSIONS_DIR}/{SOLVER_NAME}.tsv", sep="\t", header=False, index=False)

Now upload the solution from the [solutions](/solutions) folder to the Latency Challenge website.

### Optimized Solver
The following code implements an optimized solver to select the best connections for maximum profit.

In [ ]:

import heapq

def calculate_profit(latency, messages, max_latency=10000):
    """
    Calculate profit for a given latency and number of messages.
    """
    return max(0, max_latency - latency) * messages

def optimize_connections(data_set):
    """
    Optimize connections to maximize profit while minimizing costs.
    """
    # Extract unique cities
    cities = set(data_set["city1"]) | set(data_set["city2"])
    city_to_index = {city: i for i, city in enumerate(cities)}

    # Create a priority queue for edges with profit-to-cost ratio
    edges = []
    for _, row in data_set.iterrows():
        if row["latency"] > 0 and row["cost"] > 0:
            profit = calculate_profit(row["latency"], row["messages"])
            heapq.heappush(edges, (-profit / row["cost"], profit, row["cost"], row["city1"], row["city2"]))

    # Kruskal's algorithm setup
    parent = {city: city for city in cities}

    def find(city):
        if parent[city] != city:
            parent[city] = find(parent[city])
        return parent[city]

    def union(city1, city2):
        root1 = find(city1)
        root2 = find(city2)
        if root1 != root2:
            parent[root2] = root1

    # Select edges to lease
    leased_edges = set()
    total_cost = 0

    while edges:
        _, profit, cost, city1, city2 = heapq.heappop(edges)
        if find(city1) != find(city2):
            leased_edges.add((city1, city2))
            union(city1, city2)
            total_cost += cost

    return leased_edges, total_cost


### Running the Optimized Solver
This cell runs the optimized solver and outputs the leased edges to a file.

In [ ]:

leased_edges, total_cost = optimize_connections(data_set)

# Output the leased edges as a TSV file
submission_file = f"{SUBMISSIONS_DIR}/optimized_submission.tsv"
with open(submission_file, "w") as f:
    f.write("city1\tcity2\n")  # Add header
    for city1, city2 in leased_edges:
        f.write(f"{city1}\t{city2}\n")

print(f"Submission saved to {submission_file}")